Load dependancies

In [5]:
!pip install -e ../gpxpy
import gmaps.datasets
import gmaps
from gmaps.geotraitlets import Longitude
from progiter import ProgIter as prog
from strava_export_gpx import main
import pandas as pd
import gpxpy

Obtaining file:///home/watermarkhu/projects/strava_mapper/gpxpy
  Attempting uninstall: gpxpy
    Found existing installation: gpxpy 1.4.2
    Uninstalling gpxpy-1.4.2:
      Successfully uninstalled gpxpy-1.4.2
  Running setup.py develop for gpxpy


Fix TCX and GZ files

In [6]:
folder = "/mnt/d/Users/water_000/Downloads/export_25412727/"
files = main(folder, folder + "activities", folder + "activities.csv")

Reading '/mnt/d/Users/water_000/Downloads/export_25412727/activities.csv'
Found 72 activities 
Unzipping ...
Progress: 
Normalizing ...
activities/2453671243.gpx               | Run          | Morning Run
activities/2459470423.gpx               | Run          | Morning Run
activities/2497042755.gpx               | Snowboard    | Morning Activity
activities/2514112484.gpx               | Run          | Lunch Run
activities/2519401245.gpx               | Run          | Morning Run
activities/2522229121.gpx               | Run          | Morning run
activities/2706238445.gpx               | Run          | Morning Run
activities/2712950931.gpx               | Run          | Lunch Run
activities/2712959895.gpx               | Run          | Morning Run
activities/2717923914.gpx               | Run          | Running
activities/2722525232.gpx               | Run          | Running
activities/2733023399.gpx               | Run          | Morning Run
activities/2733023547.gpx               | R

Save data to Dataframe

In [7]:
data = []
for filename, activity_type in prog(files.items()):
    with open(folder + filename, "r") as file:
        gpxdata = gpxpy.parse(file)

    init_time = gpxdata.tracks[0].segments[0].points[0].time
    for track in gpxdata.tracks:
        for segment in track.segments:
            for point in segment.points:
                if point.latitude and point.longitude:
                    data.append(dict(
                        latitude=point.latitude,
                        longitude=point.longitude,
                        time=point.time,
                        type=activity_type,
                        file=filename
                    ))

df = pd.DataFrame(data)


 72/72... rate=4.43 Hz, eta=0:00:00, total=0:00:16, wall=00:04 CET


Plot map

In [8]:
with open("api_key", "r") as file:
    api_key = file.read()
gmaps.configure(api_key=api_key)
# locations = df.loc[df["type"] == "Alpine Ski", ['latitude', 'longitude']]
# locations = df.loc[df["type"] == "Ride", ['latitude', 'longitude']]
locations = df.loc[df["type"] == "Run", ['latitude', 'longitude']]
fig = gmaps.figure(
    styles='''[
  {
    "elementType": "geometry",
    "stylers": [
      {
        "color": "#f5f5f5"
      }
    ]
  },
  {
    "elementType": "labels",
    "stylers": [
      {
        "visibility": "off"
      }
    ]
  },
  {
    "elementType": "labels.icon",
    "stylers": [
      {
        "visibility": "off"
      }
    ]
  },
  {
    "elementType": "labels.text.fill",
    "stylers": [
      {
        "color": "#616161"
      }
    ]
  },
  {
    "elementType": "labels.text.stroke",
    "stylers": [
      {
        "color": "#f5f5f5"
      }
    ]
  },
  {
    "featureType": "administrative.land_parcel",
    "elementType": "labels.text.fill",
    "stylers": [
      {
        "color": "#bdbdbd"
      }
    ]
  },
  {
    "featureType": "administrative.neighborhood",
    "stylers": [
      {
        "visibility": "off"
      }
    ]
  },
  {
    "featureType": "poi",
    "elementType": "geometry",
    "stylers": [
      {
        "color": "#eeeeee"
      }
    ]
  },
  {
    "featureType": "poi",
    "elementType": "labels.text",
    "stylers": [
      {
        "visibility": "off"
      }
    ]
  },
  {
    "featureType": "poi",
    "elementType": "labels.text.fill",
    "stylers": [
      {
        "color": "#757575"
      }
    ]
  },
  {
    "featureType": "poi.business",
    "stylers": [
      {
        "visibility": "off"
      }
    ]
  },
  {
    "featureType": "poi.park",
    "elementType": "geometry",
    "stylers": [
      {
        "color": "#e5e5e5"
      }
    ]
  },
  {
    "featureType": "poi.park",
    "elementType": "labels.text.fill",
    "stylers": [
      {
        "color": "#9e9e9e"
      }
    ]
  },
  {
    "featureType": "road",
    "elementType": "geometry",
    "stylers": [
      {
        "color": "#ffffff"
      }
    ]
  },
  {
    "featureType": "road",
    "elementType": "labels.icon",
    "stylers": [
      {
        "visibility": "off"
      }
    ]
  },
  {
    "featureType": "road.arterial",
    "elementType": "labels.text.fill",
    "stylers": [
      {
        "color": "#757575"
      }
    ]
  },
  {
    "featureType": "road.highway",
    "elementType": "geometry",
    "stylers": [
      {
        "color": "#dadada"
      }
    ]
  },
  {
    "featureType": "road.highway",
    "elementType": "labels.text.fill",
    "stylers": [
      {
        "color": "#616161"
      }
    ]
  },
  {
    "featureType": "road.local",
    "elementType": "labels.text.fill",
    "stylers": [
      {
        "color": "#9e9e9e"
      }
    ]
  },
  {
    "featureType": "transit",
    "stylers": [
      {
        "visibility": "off"
      }
    ]
  },
  {
    "featureType": "transit.line",
    "elementType": "geometry",
    "stylers": [
      {
        "color": "#e5e5e5"
      }
    ]
  },
  {
    "featureType": "transit.station",
    "elementType": "geometry",
    "stylers": [
      {
        "color": "#eeeeee"
      }
    ]
  },
  {
    "featureType": "water",
    "elementType": "geometry",
    "stylers": [
      {
        "color": "#c9c9c9"
      }
    ]
  },
  {
    "featureType": "water",
    "elementType": "labels.text.fill",
    "stylers": [
      {
        "color": "#9e9e9e"
      }
    ]
  }
]''')
fig.add_layer(gmaps.heatmap_layer(locations))
fig

# %%


Figure(layout=FigureLayout(height='420px'))